In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/GST-Tacotron-master

/content/drive/MyDrive/GST-Tacotron-master


In [3]:
pip install pypinyin

     |████████████████████████████████| 1.3 MB 5.2 MB/s 


In [4]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 239 kB 5.2 MB/s 
     |████████████████████████████████| 150 kB 55.0 MB/s 
     |████████████████████████████████| 386 kB 57.2 MB/s 
     |████████████████████████████████| 64 kB 2.9 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 59 kB 6.5 MB/s 
     |████████████████████████████████| 176 kB 57.3 MB/s 
     |████████████████████████████████| 89 kB 7.6 MB/s 
     |████████████████████████████████| 1.5 MB 44.1 MB/s 
     |████████████████████████████████| 98 kB 7.4 MB/s 
     |████████████████████████████████| 12.6 MB 42.3 MB/s 
     |████████████████████████████████| 138 kB 46.3 MB/s 
     |████████████████████████████████| 1.5 MB 43.2 MB/s 
     |████████████████████████████████| 1.4 MB 46.1 MB/s 
     |████████████████████████████████| 4.9 MB 43.8 MB/s 
ERROR: Could not find a version that satisfies the requirement opencv-python==3.4.0.12 (from versions: 3.4.0.14, 3.4.2.17, 3.4.3.18, 3.4.4.1

In [12]:
from utils import *
from Data import get_eval_data
from Hyperparameters import Hyperparameters as hp
import torch
from scipy.io.wavfile import write
from Network import *

from pypinyin import lazy_pinyin, Style

device = torch.device('cpu')

def spectrogram2wav(mag):
    '''# Generate wave file from spectrogram'''
    # transpose
    #print(mag.shape, "Mag shape")
    mag = mag.T

    # de-noramlize
    mag = (np.clip(mag, 0, 1) * hp.max_db) - hp.max_db + hp.ref_db

    # to amplitude
    mag = np.power(10.0, mag * 0.05)

    # wav reconstruction
    wav = griffin_lim(mag)

    # de-preemphasis
    wav = signal.lfilter([1], [1, -hp.preemphasis], wav)

    # trim
    wav, _ = librosa.effects.trim(wav)
    #print(mag.shape)
    return wav.astype(np.float32)


def griffin_lim(spectrogram):
    '''Applies Griffin-Lim's raw.
    '''
    X_best = copy.deepcopy(spectrogram)
    for i in range(hp.n_iter):
        X_t = invert_spectrogram(X_best)
        est = librosa.stft(X_t, 2*hp.n_fft, hp.hop_length, win_length=hp.win_length)
        #print(est.shape, "Est shape")
        phase = est / np.maximum(1e-8, np.abs(est))
        #print(phase.shape, "Phase shape")
        X_best = spectrogram * phase
    X_t = invert_spectrogram(X_best)
    y = np.real(X_t)

    return y


def synthesis(model, eval_text):
    eval_text = _pinyin(eval_text)

    model.eval()

    # ref_wavs = [
    #     'ref_wav/nannan.wav', 'ref_wav/xiaofeng.wav', 'ref_wav/donaldduck.wav'
    # ]
    ref_wavs = [
        'ref_wav/nannan.wav',
        'ref_wav/xiaofeng.wav',
        'ref_wav/donaldduck.wav'
    ]
    speakers = ['nannan', 'xiaofeng', 'donaldduck']

    wavs = {}

    for ref_wav, speaker in zip(ref_wavs, speakers):
        text, GO, ref_mels = get_eval_data(eval_text, ref_wav)
        text = text.to(device)
        GO = GO.to(device)
        ref_mels = ref_mels.to(device)

        mel_hat, mag_hat, attn = model(text, GO, ref_mels)
        mag_hat = mag_hat.squeeze().detach().cpu().numpy()
        attn = attn.squeeze().detach().cpu().numpy()
        #print(mag_hat.shape)
        wav_hat = spectrogram2wav(mag_hat)
        wavs[speaker] = wav_hat

    return wavs


def load_model(checkpoint_path):
    model = Tacotron().to(device)
    model.load_state_dict(
        torch.load(
            checkpoint_path, map_location=lambda storage, location: storage))
    return model


def _pinyin(s):
    symbols = '0123456789abcdefghijklmnopqrstuvwxyz '
    s = lazy_pinyin(s, style=Style.TONE2)
    yin = []
    for token in s:
        if token != ' ':
            a = ''
            for c in token:
                if c in symbols:
                    a += c
            yin.append(a)
    a = ''
    s = ' '.join(yin)
    for i in range(len(s)):
        if s[i] == ' ' and i < len(s) - 1 and s[i + 1] == ' ':
            continue
        a += s[i]
    return a



In [16]:
text = '''毛主席是中国的红太阳'''
text = "刘易斯汉密尔顿赢得摩纳哥大奖赛"
text = "太阳从东方升起"
model = load_model('checkpoint/epoch100.pt')
wavs = synthesis(model, text)
for k in wavs:
  wav = wavs[k]
  write('samples/{}.wav'.format(k), hp.sr, wav)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [ ]:
len("PE abcdefghijklmnopqrstuvwxyz'.?")